# 概要
很多机器学习问题中每个实例都有成千上万的特征，不仅导致训练很慢，也很难找到合适的模型。这种情况称为维数灾难curse of dimennsionally。  
幸运的是，现实世界中，通常都能通过考虑减少特征数量，从而把事情简单化。比如我们的MNIST图像，边缘像素通常都是白色的，可以完全不考虑这些像素，而且也不怎么会丢失信息。而邻近的两个像素通常是高关联的，可以整合成一个单独的像素。  
降维确实会丢失一部分信息(就像把图片转换成JPEG会损失品质一样)，虽然加快了训练速度，可能会使系统变糟。这也会使流水线复杂和难以管理，所以首先要在不降维时尝试训练系统除非实在太慢。多数情况下，降维能减少数据噪声和一些不重要的细枝末节。  
除了加速训练，降维还应用于数据可视化，降到2或3维能够直接用图来表示数据关系，经常能得到一些模式比如簇。  
降维的两个主要方法projection聚合和Mainfold learning流行学习。 
3个主要算法:pca,kernel pca,和lle。
# 维度灾难
很多事物在高维空间的行为很不能理解。比如，在一个1*1的空间中选中一个随机的点，它有大概百分0.4的概率落在其边界。而对于10000D的超立体，这个概率高达99.999999%。  
还有高维中两点的距离就变得很远。  
原理上对于维度灾难，就要选择很大的高密度的训练集去训练，但是这种要求对维数是指数级增长，
# 主要的降维思路
## projection聚合
对于大多数现实问题，训练集是不能适应全部维度的。  
很多特征几乎不变，而某些特征高度关联，所以训练集其实只在原空间的子空间里分布。所以如果将样例聚合到子空间来分析。我们将得到更少特征的数据集。然而聚合不总是最佳的途径。有的分布就像是高维空间中的流型。这个时候的聚合会使得难以区分。  
## Mainfold Learning 流型学习
流型学习基于对数据是在高维空间中弯曲的超曲平面的假设，这种假设通常从经验得来。  
流型学习还要在转换为流型时，数据集会变得简单的情况下使用。有时这种变换会使得系统变复杂。
# PCA
Principal Component Analysis(主成分分析)是流行的降维算法，首先定义一个超平面，然后将数据都聚合在上面。
## 保留方差
首先，需要选择合适的超平面，一个合理的选择方法是选择保留了更多方差的超平面，这样才不会丢失太多信息，这种方向选择也是最小化原数据集和聚合后的数据集平方差距离的方向。
## 主成分
主成分各向量正交。使用pca方法得到的向量方向或者次序都是不稳定的，但是始终是同一个超平面。  
那么如何找出主成分呢，有个标准的矩阵分解技术：奇异值分解Singular Value Decomposition(SVD),可以将原矩阵分解成三个矩阵点乘$\mathbf U\cdot \sum\cdot \mathbf V^T$,其中$\mathbf V^T$含有我们寻找的主成分  
$\mathbf V^T=
\left(\begin{array}{llll}
|&|&\quad &|\\
\mathbf c_1&\mathbf c_2&\cdots&\mathbf c_n\\
|&|&\quad &|\\
\end{array}
\right)
$  
numpy有专门的函数，可以用如下代码获得：  
X_centered = X - X.mean(axis=0)  
U, s, V = np.linalg.svd(X_centered)  
c1 = V.T[:,0]  
c1 = V.T[:,1]  
PCA假设数据集按照原点对称分布，所以在使用sklearn的PCA方法时要先将数据中心化。

## 聚合到D维
一旦得到主成分，就能把原维度降低到d维。简单的将训练矩阵$\mathbf X$点乘矩阵$\mathbf W_d$(含有前d个主成分，可以由$\mathbf V^T$分解得到)：   
$\mathbf X_{d-proj}=\mathbf X\cdot\mathbf W_d$  
## 使用sklearn  
sklearn 有专门的PCA类,自动中心化：  
from sklearn.decomposition import PCA  
pca = PCA(n_components=2)  
X2D = pca.fit_transform(X)
## 可解释的方差率
一个有效的信息是每个主成分的explained variance ratio。它表示每个主成分方向方差占总方差的比例。
## 选择合适的维数
最佳维度是要包含原方差的百分95.除非是为了数据可视化，这通常是2-3D  
可以计算最少维度能够达到95%的维度数目：
pca = PCA()  
pca.fit(X)
cumsum = np.cumsum(pca.explained_variance_ratio_)  
d = np.argmax(cunsum >= 0.95) + 1  
也可以设置n_comonents为0-1的数值，这就是需要的比例总和  
pca = PCA(n_components=0.95)  
X_reduced = pca.fit_transform(X)  
还有个方法就是绘制维数和方差占比曲线  
## PCA for 压缩
显然随着维数降低，数据集变小，对于MNIST，可以吧784个特征减小至150个。这是一个合理的压缩比例，大大加速了分类算法的训练。  
也可以通过解压缩，使用PCA的逆变换pca.inverse_transform()还原数据集维度，但是肯定不能得到原始数据集，因为损失了大约百分5的方差。  
原始数据和还原数据的平方差距离称为reconstruction error重建误差。  
重建方程如下：   
$\mathbf X_{recovered}=\mathbf X_{d-proj}\cdot \mathbf W_d^T$  
# 递增pca
一个PCA的问题是在使用SVD算法的时候，所有数据都需要加载到内存中。一个解决办法是IPCA(Incremental PCA)可以将数据集分为若干个mini-batch并反馈到IPCA算法，这使得OnelinePCA得以实现。  
from sklearn.decomposition import IncrementalPCA   
n_batches = 100    
inc_pca = IncrementalPCA(n_components=154)  
for X_batch in np.array_split(X, n_batches):  
    inc_pca.partial_fit(X_batch)  
X_reduced = inc_pca.transform(X)  
还可以使用np的memmap类，可以将内存不足以加载的储存在二进制文件中的数组，仅加载其中需要在内存中加载的数据。这就使得IPCA时任意时刻只使用一部分数据，而且内存使用还能得到控制，可以使用fit()  
X_mm = np.menmap(filename, dtype="float32",mode="readonly",shape=(m,n))  
batch_size = m  
inc_pca = IncrementalPCA(n_components=154,batch_size=batch_size)  
inc_pca.fit(X_mm)

# 随机PCA
sklearn还提供其他的PCA选项，叫做随机PCA(Randomized PCA)。这是一种随机算法，能够快速找到前d个主成分，它的计算复杂度是$O(m\times d^2)+O(d^3)$,而不是PCA的$O(m\times n^2)+O(n^3)$，所以当d比n小很多的时候，执行非常快速。  
rnd_pca = PCA(n....,svd_solver="randomized")
# Kernel PCA
核技巧可以应用在PCA，可以处理复杂的非线性聚合，聚合后能够保持原数据集的某些模式，甚至还能处理流型学习。  
from sklearn.decomposition import KernelPCA  
rbf_pca = KernelPCA(n...,kernel="rbf",gamma=0.04)   
X_reduced = rbf_pca.fit...  
## 选择核及调参  
kPCA是一种非监督学习算法，没有明显的选择核函数和调参办法。然而，降维时监督学习的一个准备步骤，所以可以通过GridSearch结合后面的训练模型来寻找合适的参数。  
另一种途径，使通过最小化的重建损失来抉择。  
如果特征空间是无穷维度的，那么便无法计算重建损失，幸运的是，可以找到一个原空间的点，对应重建空间的点，这叫做预图形pre-image。可以由此计算重建后的点和原空间点的距离，然后最小化这个距离来寻找合适的核函数和参数。这个方法可以使用如下代码实现  
rbf_pca = KernelPCA(n_components=2,kernel="rbf",gamma=0.0433,fit_inverse_transform=True)  
X_reduced = rbf_pca.fit_transform(X)  
X_preimage = rbf_pca.inverse_transform(X_reduced)  
然后就能通过CV和GS来寻找合适参数。
# LLE
Locally Linear Embedding本地线性植入也是一个有力的非线性维度下降技术。这是一个流型学习技术而不是聚合。LLE先计算每个样例与其最近邻的线性关系，然后寻找能够低维表示而不损失本地关系的表达。这对于流行学习十分有效，特别是在噪声不强的情况下。  
下面是LLE的原理:首先，对于每个样例$\mathbf x^{(i)}$,算法找到其最近的k个近邻点，然后尝试重建$\mathbf x^{(i)}$使得它和近邻们保持线性关系。特别地，寻找合适的$\omega_{i,j}$使得$\mathbf x^{(i)}$和$\sum_{j=1}^m\omega_{i,j}\mathbf x^{(j)}$的平方差距离最小，假设当$\mathbf x^{(j)}$不是$\mathbf x^{(i)}$的k近邻时$\omega_{i,j}=0$。这样便得到了一个矩阵$\mathbf W$包含了各个权值$\omega_{i,j}$,然后就是简单的规范化权值。  
$\hat{\mathbf W}=argmin_{\mathbf W}\sum_{i=1}^m||x^{(i)}-\sum_{j=1}^m\omega_{i,j}\mathbf x^{(j)}||^2$  
$\text{subject to }
\left{
\begin{array}{ll}
\omega_{i,j}=0&\text{if $\mathbf x^{(j)}$is not one of the k c.n of $\mathbf x^{(i)}$}\\
\sum_{j=1}^m\omega_{i,j}=1&\text{for$i=1,2,\cdots,m$}\\
\end{array}
\right.
$  
这步完成后将权重矩阵作为样例之间的线性关系。下一步，尽可能多的保存这些线性关系的条件下将训练集映射到d维空间，如果$\mathbf z^{(i)}$是$\mathbf x^{(i)}$在d维空间的像，我们就是希望$\mathbf z^{(i)}$与$\sum_{j=1}^m\hat\omega_{i,j}\mathbf z^{(j)}$的平方差尽可能的小。这种思想用表达式如下:

$\hat{\mathbf Z}=argmin_{\mathbf Z}\sum_{i=1}^m||\mathbf z^{(i)}-\sum_{j=1}^m\hat\omega_{i,j}\mathbf z^{(j)}||^2$  
这和第一步非常相似，但是不是为了最小距离而找最优权重，而是在低维空间中保留权重，找到最合适的样例的像的位置。  
sklearn中LLE的计算复杂度是寻找k近邻使用$O(m log(m)n log(k))$，最优化权重$O(mnk^3)$,重建低维表示$O(dm^2)$,不幸的是，最后项m的平方使得算法在大数据集情况下难以使用、
# 其他降维技术
sklearn还有许多降维算法，常用的如下  

Multidimensional Scaling(MDS)在降维时，保持样例的距离。  
Isomap将样例和它的临近点用图连接起来，降维时尽量保持geodesic distance，即两节点最短路径的节点数目。  
t-Distributed Stochastic Neighbor Embedding(t-SNE)t分布随机近邻植入，降维时尽量保持相似样例接近，不同的样例保持分隔，通常用于可视化，特别是高维空间的可视化簇。  
Linear Discriminant Analysis(LDA)线性判别式分析，实际上是一个分类算法，在训练时学习类之间的判别中心轴，然后使用这些轴来聚合数据，这个好处是可以使类之间保持距离，所以LDA是在进行分类算法比如SVM前降维的一个优秀算法。